In [ ]:
%pip install -q -r requirements.txt

In [ ]:
import importlib
import functions.core.core_functions as core_functions
import dask
import dask.dataframe as dd
from dask.dataframe.utils import assert_eq
import pandas as pd
import numpy as np
import pandas_gbq
import json
import gc
import cudf
import os
from datetime import datetime, timedelta

dask.config.set({"dataframe.backend": "cudf"})

importlib.reload(core_functions)

In [ ]:
init = {}
init = core_functions.initialize_clients(service_account_secret_name='SA_N90_CORE_APPS')

config = init.get('config')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_N90_CORE_APPS')
bigquery_client = init.get('clients').get('bigquery_client')
storage_client = init.get('clients').get('storage_client')

In [ ]:
client_gads_accounts = f"""
select distinct system_account_id from `next90-core-applications.next90_analytics.client_systems` 
where system_platform = 'GoogleAds'
"""
gads_ids = list(core_functions.fetch_gbq_data(client_gads_accounts, bigquery_client).values.flatten().astype(int))
gads_ids


In [ ]:
gclid_sql = f"""
  CREATE TEMP TABLE sessionGclids AS (
    with gclidBase AS (
select gclid as click_gclid, min(created_time) as click_timestamp from `next90-core-applications.omniData.activity_sessions` 
where gclid is not null
group by gclid
)
select click_gclid, click_timestamp, id as click_session_id,
        DATE(click_timestamp) AS click_date
from gclidBase
join `next90-core-applications.omniData.activity_sessions` 
on click_gclid = gclid
and click_timestamp = created_time
order by click_timestamp
  );
SELECT * FROM `next90-core-applications.next90_google_ads_transfer_new.ads_ClickStats_8417741864`
left join sessionGclids
on click_gclid = click_view_gclid
AND (click_date = segments_date or click_date = DATE_ADD(segments_date, INTERVAL 1 DAY))
WHERE customer_id in ({','.join([str(x) for x in gads_ids])})
"""

gclid_base_df = core_functions.fetch_gbq_data(gclid_sql, bigquery_client=bigquery_client)
gclid_base_df_working = gclid_base_df.copy()

In [ ]:
# gclid_base_df_working = gclid_base_df.copy()


In [ ]:
gclid_base_df_working['date'] = gclid_base_df_working['segments_date']
gclid_base_df_working = core_functions.extract_year_month_day(gclid_base_df_working, 'date')

In [ ]:
call_rail_sql = f"""
SELECT * FROM `next90-core-applications.next90_analytics.callRailCalls`
"""

call_rail_base_df = core_functions.fetch_gbq_data(call_rail_sql, bigquery_client=bigquery_client)

In [ ]:
call_rail_base_df['call_rail_id'] = call_rail_base_df['id']


In [ ]:
call_rail_base_df['call_gclid'] = call_rail_base_df['gclid']
call_rail_base_df['call_timestamp'] = call_rail_base_df['start_time_utc']
call_rail_base_df['call_session_id'] = call_rail_base_df['id']

In [ ]:
call_rail_base_df

In [ ]:
call_rail_gclid_times_df = call_rail_base_df.loc[(call_rail_base_df['gclid'].notnull()) & (call_rail_base_df['gclid'] != 'None'),['call_gclid','call_timestamp','call_session_id']]

In [ ]:
call_rail_gclid_times_df.sort_values('call_timestamp', ascending=False, inplace=True)
call_rail_gclid_times_df.drop_duplicates('call_gclid', keep='first', inplace=True)

In [ ]:
gclid_base_df_working = gclid_base_df_working.merge(call_rail_gclid_times_df, how='left', left_on='click_view_gclid', right_on='call_gclid', suffixes=('_click', '_call'))


In [ ]:
gclid_base_df_working.loc[(gclid_base_df_working['call_timestamp'].notnull()) & (gclid_base_df_working['click_timestamp'].isna())]

In [ ]:
core_functions.show_more_dataframe()
gclid_base_df_working['kw_criteria_id'] = gclid_base_df_working['click_view_keyword'].str.split('~').str[1]
gclid_base_df_working['ad_id'] = gclid_base_df_working['click_view_ad_group_ad'].str.split('~').str[1]
gclid_base_df_working['click_city_presence_location'] = gclid_base_df_working['click_view_location_of_presence_city'].str.split('/').str[1]
gclid_base_df_working['click_metro_presence_location'] = gclid_base_df_working['click_view_location_of_presence_metro'].str.split('/').str[1]
gclid_base_df_working['click_most_specific_presence_location'] = gclid_base_df_working['click_view_location_of_presence_most_specific'].str.split('/').str[1]
gclid_base_df_working['view_timestamp'] = (gclid_base_df_working['click_timestamp'] - timedelta(seconds=1)).fillna(gclid_base_df_working['call_timestamp'] - timedelta(seconds=1))
gclid_base_df_working.loc[~gclid_base_df['click_view_keyword'].isna()].head()

In [ ]:
gclid_base_df_working.loc[gclid_base_df_working['view_timestamp'].notnull()]

In [ ]:
gclid_base_df_working.loc[~gclid_base_df['click_view_keyword'].isna()].head()
min_date = gclid_base_df_working['date'].min().date()
min_date


In [ ]:
gclid_base_df_working.head(3)

In [ ]:
kw_stats_sql = f"""
SELECT * FROM `next90-core-applications.next90_google_ads_transfer_new.ads_KeywordStats_8417741864` 
WHERE segments_date >= '{min_date}'
AND customer_id IN ({','.join([str(x) for x in gads_ids])})
"""

kw_stats_df = core_functions.fetch_gbq_data(kw_stats_sql, bigquery_client=bigquery_client)
kw_stats_df.head(3)


In [ ]:
kw_stats_df['ad_group_criterion_criterion_id'] = kw_stats_df['ad_group_criterion_criterion_id'].astype(str)

In [ ]:
for col in gclid_base_df_working.columns:
    if col in kw_stats_df.columns:
        print(col)

In [ ]:
gclid_merge_fields = ['kw_criteria_id', 'ad_group_id', 'campaign_id', 'customer_id', 'segments_date', 'segments_ad_network_type', 'segments_click_type', 'segments_device']
kw_stats_merge_fields = ['ad_group_criterion_criterion_id', 'ad_group_id', 'campaign_id', 'customer_id', 'segments_date', 'segments_ad_network_type', 'segments_click_type', 'segments_device']

In [ ]:
gclid_base_df_working2 = gclid_base_df_working.merge(kw_stats_df, how='left', left_on=gclid_merge_fields, right_on=kw_stats_merge_fields, suffixes=('', '_kw_stats'))

In [ ]:
gclid_base_df_working2.loc[~gclid_base_df_working2['metrics_average_cpc'].isna()].head()

In [ ]:
search_query_stats_sql = f"""
SELECT * FROM `next90-core-applications.next90_google_ads_transfer_new.ads_SearchQueryStats_8417741864` 
WHERE segments_date >= '{min_date}'
AND customer_id IN ({','.join([str(x) for x in gads_ids])})
"""

search_query_stats_df = core_functions.fetch_gbq_data(search_query_stats_sql, bigquery_client=bigquery_client)
search_query_stats_df.head(3)

In [ ]:
search_query_stats_df['kw_criteria_id'] = search_query_stats_df['segments_keyword_ad_group_criterion'].str.split('~').str[1]

In [ ]:
search_query_stats_df.head(3)

In [ ]:
col_list = []
for col in gclid_base_df_working2.columns:
    if col in search_query_stats_df.columns and 'metrics' not in col and not col.endswith('_DATE'):
        print(col)
        col_list.append(col)

In [ ]:
gclid_base_df_working3 = gclid_base_df_working2.merge(search_query_stats_df, how='left', left_on=col_list, right_on=col_list, suffixes=('', '_search_query_stats'))

In [ ]:
gclid_base_df_working3.sort_values(['view_timestamp', 'metrics_active_view_impressions','metrics_average_cpc','metrics_ctr','metrics_impressions_search_query_stats' ], ascending=False, inplace=True)

In [ ]:
gclid_base_df_working3 = gclid_base_df_working3.drop_duplicates(subset=['click_view_gclid']).sort_values('view_timestamp', ascending=True)

In [ ]:
# paid_organic_sql = f"""
# SELECT * FROM `next90-core-applications.next90_google_ads_transfer_new.ads_PaidOrganicStats_8417741864` 
# WHERE segments_date >= '{min_date}'
# AND customer_id IN ({','.join([str(x) for x in gads_ids])})
# """
# paid_organic_df = core_functions.fetch_gbq_data(paid_organic_sql, bigquery_client=bigquery_client)

In [ ]:

# for col in paid_organic_df.columns:
#     if 'segments' in col.lower():
#         print(col)

In [ ]:
# paid_organic_df
# ad_group_id , campaign_id , customer_id , paid_organic_search_term_view_search_term , segments_date 

In [ ]:
gclid_base_df_working3.loc[(gclid_base_df_working3['segments_click_type'] == 'CALLS') & (~gclid_base_df_working2['view_timestamp'].isna())]

In [ ]:

# n90_storage_options = None
n90_storage_options = config.get('N90_GCS_STORAGE_OPTIONS')

# process_df['profile__attributes']
n90_bucket = None
n90_bucket = 'n90-data-lake'
n90_output_prefix = None
n90_output_prefix = 'analitics/google-ads-gclid-details'
partition_cols = None
partition_cols = ['year', 'month', 'day']

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_N90_CORE_APPS')
core_functions.write_hive_partitioned_parquet(gclid_base_df_working2, n90_bucket, n90_output_prefix, partition_cols, n90_storage_options)
print(f"Finished writing to {n90_bucket}/{n90_output_prefix}")


In [ ]:
sixty_sec_gclid_calls_sql = f"""
select id from `next90-core-applications.next90_analytics.callRailCalls` where gclid is not null and gclid != 'None' and duration >= 60
"""
sixty_sec_gclid_calls_df = core_functions.fetch_gbq_data(sixty_sec_gclid_calls_sql, bigquery_client=bigquery_client)
sixty_sec_call_ids = list(sixty_sec_gclid_calls_df['id'].values)

In [ ]:
today = datetime.today()
date_90_days_ago = (today - timedelta(days=90)).date()
date_90_days_ago_str = date_90_days_ago.strftime('%Y-%m-%d')

In [ ]:
date_90_days_ago

In [ ]:
gclid_call_conv_df = gclid_base_df_working3.loc[gclid_base_df_working3['call_session_id'].isin(sixty_sec_call_ids)].copy()

In [ ]:
def format_timestamp_no_subseconds(t):
    if pd.isnull(t):
        return ''
    return t.strftime('%Y-%m-%d %H:%M:%S%z')[:-2] + ':' + t.strftime('%Y-%m-%d %H:%M:%S%z')[-2:]


In [ ]:
# gclid_call_conv_df['call_timestamp'] = pd.to_datetime(gclid_call_conv_df['call_timestamp'], utc=True)
gclid_call_conv_df['call_timestamp_prepped'] = gclid_call_conv_df['call_timestamp'].apply(
    lambda x: x.isoformat(sep=' ') if pd.notnull(x) else ''
)
gclid_call_conv_df['call_timestamp_prepped'] = gclid_call_conv_df['call_timestamp'].apply(format_timestamp_no_subseconds)
gclid_call_conv_df.loc[gclid_call_conv_df['segments_date'] > date_90_days_ago]
gclid_call_conv_df['customer_id_str'] = gclid_call_conv_df['customer_id'].astype(str)
upload_records = gclid_call_conv_df.loc[(gclid_call_conv_df['segments_date'] > date_90_days_ago) & (gclid_call_conv_df['customer_id'] == 6111416100),['click_view_gclid','customer_id_str','call_timestamp_prepped', 'call_session_id']][2:10].to_dict(orient='records')

In [ ]:
upload_records

In [ ]:
from google.ads.googleads.client import GoogleAdsClient
from google.ads.googleads.errors import GoogleAdsException

googleads_client = GoogleAdsClient.load_from_storage( )

def get_customer_info(client, customer_id):
    ga_service = client.get_service("GoogleAdsService", version="v18")
    # query = "SELECT campaign.name FROM campaign LIMIT 10"
    query = """
        SELECT
          campaign.id,
          campaign.name
        FROM campaign
        ORDER BY campaign.id"""
    response = ga_service.search_stream(customer_id=customer_id, query=query)
    # Access the iterator in the same scope as where the service object was created.
    try:
        for batch in response:
            for row in batch.results:
                print(
                    f"Campaign with ID {row.campaign.id} and name "
                    f'"{row.campaign.name}" was found.'
                )
    except GoogleAdsException as ex:
        
            
# def get_customer_info(client, customer_id):
#     ga_service = client.get_service("GoogleAdsService")

#     query = """
#         SELECT
#           campaign.id,
#           campaign.name
#         FROM campaign
#         ORDER BY campaign.id"""

#     # Issues a search request using streaming.
#     result = ga_service.search(customer_id=customer_id, query=query)

    # for batch in stream:
    #     for row in batch.results:
    #         print(
    #             f"Campaign with ID {row.campaign.id} and name "
    #             f'"{row.campaign.name}" was found.'
    #         )
            


In [ ]:
def get_conversion_actions(client, customer_id):
    ga_service = client.get_service("GoogleAdsService", version="v18")
    # query = "SELECT campaign.name FROM campaign LIMIT 10"
    query = """
        SELECT
  conversion_action.id,
  conversion_action.name,
  conversion_action.status,
  conversion_action.category,
  conversion_action.type,
  conversion_action.owner_customer,
  conversion_action.resource_name
FROM
  conversion_action"""
    response = ga_service.search_stream(customer_id=customer_id, query=query)
    # Access the iterator in the same scope as where the service object was created.
    try:
        for batch in response:
            for row in batch.results:
                print(
                    f"Conversion Action  ID {row.conversion_action.id} and name "
                    f'"{row.conversion_action.name}" was found.'
                )
    except GoogleAdsException as ex:
        print(ex)
        
def get_conversion_custom_variables(client, customer_id):
    ga_service = client.get_service("GoogleAdsService", version="v18")
    # query = "SELECT campaign.name FROM campaign LIMIT 10"
    query = """
        SELECT
  conversion_custom_variable.id,
  conversion_custom_variable.name,
  conversion_custom_variable.tag,
  conversion_custom_variable.status,
  conversion_custom_variable.resource_name
FROM
  conversion_custom_variable"""
    response = ga_service.search_stream(customer_id=customer_id, query=query)
    # Access the iterator in the same scope as where the service object was created.
    try:
        for batch in response:
            for row in batch.results:
                print(
                    f"Conversion Custom Variable  ID {row.conversion_custom_variable.id} and name "
                    f'"{row.conversion_custom_variable.name}" was found.'
                )
    except GoogleAdsException as ex:
        print(ex)

In [ ]:
get_conversion_custom_variables(googleads_client, '6111416100')

In [ ]:
get_conversion_actions(googleads_client, '6111416100')

In [ ]:
def add_offline_click_conversion(
    client,
    customer_id,
    conversion_action_id,
    gclid,
    conversion_date_time,
    conversion_value,
    conversion_custom_variable_id,
    conversion_custom_variable_value,
    gbraid,
    wbraid,
    order_id,
    ad_user_data_consent,
):
    """Creates a click conversion with a default currency of USD.

    Args:
        client: An initialized GoogleAdsClient instance.
        customer_id: The client customer ID string.
        conversion_action_id: The ID of the conversion action to upload to.
        gclid: The Google Click Identifier ID. If set, the wbraid and gbraid
            parameters must be None.
        conversion_date_time: The the date and time of the conversion (should be
            after the click time). The format is 'yyyy-mm-dd hh:mm:ss+|-hh:mm',
            e.g. '2021-01-01 12:32:45-08:00'.
        conversion_value: The conversion value in the desired currency.
        conversion_custom_variable_id: The ID of the conversion custom
            variable to associate with the upload.
        conversion_custom_variable_value: The str value of the conversion custom
            variable to associate with the upload.
        gbraid: The GBRAID for the iOS app conversion. If set, the gclid and
            wbraid parameters must be None.
        wbraid: The WBRAID for the iOS app conversion. If set, the gclid and
            gbraid parameters must be None.
        order_id: The order ID for the click conversion.
        ad_user_data_consent: The ad user data consent for the click.
    """
    click_conversion = client.get_type("ClickConversion")
    conversion_upload_service = client.get_service("ConversionUploadService")
    conversion_action_service = client.get_service("ConversionActionService")
    click_conversion.conversion_action = (
        conversion_action_service.conversion_action_path(
            customer_id, conversion_action_id
        )
    )

    # Sets the single specified ID field.
    if gclid:
        click_conversion.gclid = gclid
        print(f"Setting gclid to {gclid}")
    elif gbraid:
        click_conversion.gbraid = gbraid
        print(f"Setting gbraid to {gbraid}")
    else:
        click_conversion.wbraid = wbraid
        print(f"Setting wbraid to {wbraid}")

    click_conversion.conversion_value = float(conversion_value)
    click_conversion.conversion_date_time = conversion_date_time
    click_conversion.currency_code = "USD"

    if conversion_custom_variable_id and conversion_custom_variable_value:
        conversion_custom_variable = client.get_type("CustomVariable")
        conversion_custom_variable.conversion_custom_variable = (
            conversion_upload_service.conversion_custom_variable_path(
                customer_id, conversion_custom_variable_id
            )
        )
        conversion_custom_variable.value = conversion_custom_variable_value
        click_conversion.custom_variables.append(conversion_custom_variable)

    if order_id:
        click_conversion.order_id = order_id

    # Sets the consent information, if provided.
    if ad_user_data_consent:
        # Specifies whether user consent was obtained for the data you are
        # uploading. For more details, see:
        # https://www.google.com/about/company/user-consent-policy
        click_conversion.consent.ad_user_data = client.enums.ConsentStatusEnum[
            ad_user_data_consent
        ]

    # Uploads the click conversion. Partial failure must be set to True here.
    #
    # NOTE: This request only uploads a single conversion, but if you have
    # multiple conversions to upload, it's most efficient to upload them in a
    # single request. See the following for per-request limits for reference:
    # https://developers.google.com/google-ads/api/docs/best-practices/quotas#conversion_upload_service
    request = client.get_type("UploadClickConversionsRequest")
    request.customer_id = customer_id
    request.conversions.append(click_conversion)
    request.partial_failure = True
    conversion_upload_response = (
        conversion_upload_service.upload_click_conversions(
            request=request,
        )
    )
    print(conversion_upload_response.results)
    uploaded_click_conversion = conversion_upload_response.results[0]
    print(
        f"Uploaded conversion that occurred at "
        f'"{uploaded_click_conversion.conversion_date_time}" from '
        f'Google Click ID "{uploaded_click_conversion.gclid}" '
        f'to "{uploaded_click_conversion.conversion_action}"'

    )

In [ ]:
for conversion in upload_records:
    add_offline_click_conversion(
        client=googleads_client,
        customer_id=str(conversion['customer_id_str']),
        conversion_action_id=269508876,
        gclid=conversion['click_view_gclid'],
        conversion_date_time=conversion['call_timestamp_prepped'],
        conversion_value=1.00,
        conversion_custom_variable_id=23124982,
        conversion_custom_variable_value='search_power',
        gbraid=None,
        wbraid=None,
        order_id=None,
        ad_user_data_consent=None
    )

In [ ]:
ga_service_2 = googleads_client.get_service("GoogleAdsService", version="v18")
gaql = """
SELECT offline_conversion_upload_conversion_action_summary.conversion_action_id, offline_conversion_upload_conversion_action_summary.conversion_action_name, offline_conversion_upload_conversion_action_summary.total_event_count, offline_conversion_upload_conversion_action_summary.successful_event_count, offline_conversion_upload_conversion_action_summary.resource_name, offline_conversion_upload_conversion_action_summary.pending_event_count, offline_conversion_upload_conversion_action_summary.job_summaries FROM offline_conversion_upload_conversion_action_summary WHERE offline_conversion_upload_conversion_action_summary.conversion_action_id = 269508876 AND customer.id = 6111416100"""
response = ga_service_2.search_stream(customer_id='6111416100', query=gaql)
    # Access the iterator in the same scope as where the service object was created.
try:
    for batch in response:
        for row in batch.results:
            print(
                row
            )
except GoogleAdsException as ex:
    print(ex)
# response = googleads_client.service.search_stream(customer_id='6111416100', query=gaql)

In [ ]:
gaql = """
SELECT
  customer.id,
  offline_conversion_upload_client_summary.alerts,
  offline_conversion_upload_client_summary.client,
  offline_conversion_upload_client_summary.daily_summaries,
  offline_conversion_upload_client_summary.job_summaries,
  offline_conversion_upload_client_summary.last_upload_date_time,
  offline_conversion_upload_client_summary.pending_event_count,
  offline_conversion_upload_client_summary.pending_rate,
  offline_conversion_upload_client_summary.status,
  offline_conversion_upload_client_summary.success_rate,
  offline_conversion_upload_client_summary.successful_event_count,
  offline_conversion_upload_client_summary.total_event_count
FROM offline_conversion_upload_client_summary
WHERE customer.id = 6111416100
"""
response = ga_service_2.search_stream(customer_id='6111416100', query=gaql)
    # Access the iterator in the same scope as where the service object was created.
try:
    for batch in response:
        for row in batch.results:
            print(
                row
            )
except GoogleAdsException as ex:
    print(ex)

In [ ]:
gaql = """
SELECT
  offline_conversion_upload_conversion_action_summary.conversion_action_name,
  offline_conversion_upload_conversion_action_summary.alerts,
  offline_conversion_upload_conversion_action_summary.client,
  offline_conversion_upload_conversion_action_summary.daily_summaries,
  offline_conversion_upload_conversion_action_summary.job_summaries,
  offline_conversion_upload_conversion_action_summary.last_upload_date_time,
  offline_conversion_upload_conversion_action_summary.pending_event_count,
  offline_conversion_upload_conversion_action_summary.status,
  offline_conversion_upload_conversion_action_summary.successful_event_count,
  offline_conversion_upload_conversion_action_summary.total_event_count
FROM offline_conversion_upload_conversion_action_summary
WHERE offline_conversion_upload_conversion_action_summary.conversion_action_id = 269508876 and customer.id = 6111416100
"""
response = ga_service_2.search_stream(customer_id='6111416100', query=gaql)
    # Access the iterator in the same scope as where the service object was created.
try:
    for batch in response:
        for row in batch.results:
            print(
                row
            )
except GoogleAdsException as ex:
    print(ex)

In [ ]:
def check_offline_user_data_job_status(client, customer_id, job_id):
    """Retrieves and prints the status of the OfflineUserDataJob with the given job ID.

    Args:
        client: An initialized GoogleAdsClient instance.
        customer_id: The client customer ID string without hyphens.
        job_id: The ID of the OfflineUserDataJob to check.
    """
    offline_user_data_job_service = client.get_service("OfflineUserDataJobService")
    resource_name = offline_user_data_job_service.offline_user_data_job_path(
        customer_id, job_id
    )

    try:
        # Retrieve the OfflineUserDataJob.
        offline_user_data_job = offline_user_data_job_service.get_offline_user_data_job(
            resource_name=resource_name
        )

        status_enum = client.enums.OfflineUserDataJobStatusEnum
        status_name = status_enum.OfflineUserDataJobStatus.Name(
            offline_user_data_job.status
        )

        print(f"OfflineUserDataJob with resource name '{resource_name}' has status: {status_name}")

        if offline_user_data_job.status == status_enum.FAILED:
            failure_reason_enum = client.enums.OfflineUserDataJobFailureReasonEnum
            failure_reason_name = failure_reason_enum.OfflineUserDataJobFailureReason.Name(
                offline_user_data_job.failure_reason
            )
            print(f"Failure reason: {failure_reason_name}")

    except GoogleAdsException as ex:
        print(
            f"Request with ID '{ex.request_id}' failed with status "
            f"'{ex.error.code().name}' and includes the following errors:"
        )
        for error in ex.failure.errors:
            print(f"\tError code: {error.error_code}")
            print(f"\tMessage: {error.message}")
            if error.location:
                for field_path_element in error.location.field_path_elements:
                    print(f"\t\tOn field: {field_path_element.field_name}")

In [ ]:
check_offline_user_data_job_status(googleads_client, '6111416100', 5773612345940294139)

In [ ]:
get_customer_info(googleads_client, '6111416100')